In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
def crps(y_true, S):
    """
    Computes continuous ranked probability score:

    Parameters
    ----------
    y_true : tf tensor of shape (BATCH_SIZE, 1)
        True values.
    S : tf tensor of shape (BATCH_SIZE, N_SAMPLES)
        Predictive samples.

    Returns
    -------
    tf tensor of shape (BATCH_SIZE,)
        Scores.

    """
    beta=1
    n_samples = S.shape[-1]
    def expected_dist(diff, beta):
        return K.sum(K.pow(K.sqrt(K.square(diff)+K.epsilon()), beta),axis=-1) #axis = -1: last dimension <=> N_SAMPLES
    es_1 = expected_dist(y_true - S, beta)
    es_2 = 0
    for i in range(n_samples):
        es_2 = es_2 + expected_dist(K.expand_dims(S[:,i]) - S, beta)
    return es_1/n_samples - es_2/(2*n_samples**2)

class CRPSLoss(Loss):
    def call(self, y_true, S):
        return crps(y_true, S)

In [4]:
""" y_pred = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]
y_true = [20.0] """
from scipy.stats import poisson

quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = [round(q, 3) for q in quantiles] # due to binary inaccuracies


mean = 50
y_pred = np.array(poisson.ppf(quantiles, mean), dtype=np.float32)
y_true = [9.0]


import CRPS.CRPS as pscore
# CRPS normal berechnen
crps_normal = pscore(np.array(y_pred), y_true).compute()[0]
print("CRPS: " + str(crps_normal))



S = tf.constant([y_pred])
print(S)
y_true = tf.constant([y_true], dtype=tf.float32)


beta=1
n_samples = S.shape[-1]

def expected_dist(diff, beta):
    return K.sum(K.pow(K.sqrt(K.square(diff)+K.epsilon()), beta),axis=-1) #axis = -1: last dimension <=> N_SAMPLES
es_1 = expected_dist(y_true - S, beta)
es_2 = 0
for i in range(n_samples):
    es_2 = es_2 + expected_dist(K.expand_dims(S[:,i]) - S, beta)

print("CRPS Näherung: " + str(es_1/n_samples - es_2/(2*n_samples**2)))

CRPS: [37.03430057]
tf.Tensor(
[[30. 31. 32. 32. 33. 33. 34. 34. 34. 34. 35. 35. 35. 35. 35. 35. 36. 36.
  36. 36. 36. 36. 36. 37. 37. 37. 37. 37. 37. 37. 37. 37. 37. 38. 38. 38.
  38. 38. 38. 38. 38. 38. 38. 38. 38. 38. 38. 39. 39. 39. 39. 39. 39. 39.
  39. 39. 39. 39. 39. 39. 39. 39. 39. 39. 40. 40. 40. 40. 40. 40. 40. 40.
  40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 40. 41. 41. 41. 41.
  41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41. 41.
  41. 41. 41. 41. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42.
  42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 42. 43.
  43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43.
  43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 43. 44.
  44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44.
  44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44. 44.
  44. 44. 44. 44. 44. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45. 45.
  45. 4